# 03 — IPI Sensitivity
Use IPI series=`abs` and `index_sa` to test contemporaneous/lag relationships with total consumption and stress.
Then fit a simple linear model with lag features and report metrics.


In [ ]:
import csv
import math
import statistics
from datetime import datetime
from pathlib import Path

DATA_DIR = Path("data/processed")
REPORTS_DIR = Path("reports")
FIG_DIR = REPORTS_DIR / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

def read_csv(path):
    with open(path, newline="", encoding="utf-8") as f:
        return list(csv.DictReader(f))

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def month_range(start, end):
    months = []
    cur = datetime(start.year, start.month, 1)
    while cur <= end:
        months.append(cur.strftime("%Y-%m-%d"))
        if cur.month == 12:
            cur = datetime(cur.year + 1, 1, 1)
        else:
            cur = datetime(cur.year, cur.month + 1, 1)
    return months


In [ ]:
# Build aligned monthly dataset
cons = {r['date']: float(r['consumption']) for r in read_csv(DATA_DIR / 'electricity_consumption_clean.csv') if r['sector'] == 'total'}
supply = {r['date']: float(r['supply']) for r in read_csv(DATA_DIR / 'electricity_supply_clean.csv') if r['sector'] == 'total'}

ipi_abs = {
    r['date']: float(r['index_sa'])
    for r in read_csv(DATA_DIR / 'ipi_clean.csv')
    if r['series'] == 'abs' and r['index_sa'].strip() != ''
}

months = sorted(set(cons).intersection(supply).intersection(ipi_abs))
rows = []
for d in months:
    ratio = cons[d] / supply[d] if supply[d] else math.nan
    rows.append({'date': d, 'consumption': cons[d], 'stress': ratio, 'ipi_abs_index_sa': ipi_abs[d]})

# stress z-score
mu = statistics.mean(r['stress'] for r in rows)
sd = statistics.pstdev(r['stress'] for r in rows)
for r in rows:
    r['stress_z'] = (r['stress'] - mu) / sd if sd else 0.0

print(f'Aligned months: {len(rows)} ({rows[0]["date"]} -> {rows[-1]["date"]})')


In [ ]:
def corr(a, b):
    ma, mb = statistics.mean(a), statistics.mean(b)
    num = sum((x-ma)*(y-mb) for x,y in zip(a,b))
    den = math.sqrt(sum((x-ma)**2 for x in a) * sum((y-mb)**2 for y in b))
    return num/den if den else 0.0

# Lag relationships: IPI leads target by k months (k=0..6)
lag_results = []
for k in range(0, 7):
    xs, yc, ys = [], [], []
    for i in range(k, len(rows)):
        xs.append(rows[i-k]['ipi_abs_index_sa'])
        yc.append(rows[i]['consumption'])
        ys.append(rows[i]['stress_z'])
    lag_results.append({
        'lag_months': k,
        'corr_ipi_to_consumption': corr(xs, yc),
        'corr_ipi_to_stress_z': corr(xs, ys),
    })

print('Lag correlation table (IPI leading)')
for r in lag_results:
    print(r)


In [ ]:
# Simple explainable model: predict consumption using IPI lags (0,1,2) with ordinary least squares

def solve_linear_system(a, b):
    # Gaussian elimination for small dense systems
    n = len(b)
    m = [row[:] + [b[i]] for i, row in enumerate(a)]
    for col in range(n):
        pivot = max(range(col, n), key=lambda r: abs(m[r][col]))
        m[col], m[pivot] = m[pivot], m[col]
        pv = m[col][col]
        if abs(pv) < 1e-12:
            continue
        for j in range(col, n+1):
            m[col][j] /= pv
        for r in range(n):
            if r != col:
                factor = m[r][col]
                for j in range(col, n+1):
                    m[r][j] -= factor * m[col][j]
    return [m[i][n] for i in range(n)]

def fit_linear_regression(X, y):
    p = len(X[0])
    xtx = [[0.0]*p for _ in range(p)]
    xty = [0.0]*p
    for row, target in zip(X, y):
        for i in range(p):
            xty[i] += row[i]*target
            for j in range(p):
                xtx[i][j] += row[i]*row[j]
    return solve_linear_system(xtx, xty)

def predict(X, beta):
    return [sum(x*b for x,b in zip(row,beta)) for row in X]

def mae(y, yhat):
    return sum(abs(a-b) for a,b in zip(y,yhat))/len(y)

def rmse(y, yhat):
    return math.sqrt(sum((a-b)**2 for a,b in zip(y,yhat))/len(y))

def r2(y, yhat):
    ybar = statistics.mean(y)
    ss_res = sum((a-b)**2 for a,b in zip(y,yhat))
    ss_tot = sum((a-ybar)**2 for a in y)
    return 1 - ss_res/ss_tot if ss_tot else 0.0

# Build lagged features
X, y = [], []
for i in range(2, len(rows)):
    X.append([1.0, rows[i]['ipi_abs_index_sa'], rows[i-1]['ipi_abs_index_sa'], rows[i-2]['ipi_abs_index_sa']])
    y.append(rows[i]['consumption'])

split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

beta = fit_linear_regression(X_train, y_train)
y_pred = predict(X_test, beta)

print('Coefficients [intercept, ipi_t, ipi_t-1, ipi_t-2]:')
print([round(v, 4) for v in beta])
print('Test metrics:')
print({'MAE': round(mae(y_test, y_pred), 3), 'RMSE': round(rmse(y_test, y_pred), 3), 'R2': round(r2(y_test, y_pred), 4), 'n_test': len(y_test)})
